<a href="https://colab.research.google.com/github/samuel-shaibu/NLP/blob/main/Chatbot_with_LSTM_for_an_Ecommerce_site_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
#Import necessary modules
import json
import numpy as np
import random
import tensorflow as tf
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [63]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [64]:
# Sample Intent
intents = {
  "intents": [
      {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good afternoon"],
        "responses": ["Hi there! How can I assist you today?", "Hello! How may I help you?"]
      },
      {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Talk to you later"],
        "responses": ["Goodbye! Have a great day!", "Thanks for visiting!"]
      },
      {
        "tag": "thanks",
        "patterns": ["Thanks", "Thank you", "Thanks a lot", "Appreciate it"],
        "responses": ["You're welcome!", "Happy to help!"]
      },
      {
        "tag": "order_status",
        "patterns": ["Where is my order?", "Track my order", "Order status", "Has my order been shipped?"],
        "responses": ["Can you please provide your order ID?", "Sure, what's your order number?"]
      },
      {
        "tag": "product_inquiry",
        "patterns": ["Do you have red sneakers?", "I want a blue t-shirt", "Show me phones under 500 dollars", "Do you sell headphones?"],
        "responses": ["Let me check that for you...", "Yes, we do! Visit the product section or search using filters."]
      },
      {
        "tag": "returns",
        "patterns": ["I want to return a product", "How do I return an item?", "Return policy", "Can I return something I bought?"],
        "responses": ["You can return any item within 30 days. Just go to your orders and click 'Return'.", "Sure! Our return policy allows returns within 30 days of delivery."]
      },
      {
        "tag": "shipping",
        "patterns": ["Do you ship internationally?", "How long does delivery take?", "What are the shipping charges?", "Shipping information"],
        "responses": ["We offer worldwide shipping. Delivery takes 3-7 days.", "Shipping is free on orders over $50."]
      },
      {
        "tag": "payment",
        "patterns": ["What payment methods do you accept?", "Can I pay with PayPal?", "Do you support credit cards?"],
        "responses": ["We accept PayPal, credit/debit cards, and more.", "Yes, we support multiple payment options including digital wallets."]
      },
      {
        "tag": "cancel_order",
        "patterns": ["How do I cancel my order?", "I want to cancel something I bought", "Cancel order"],
        "responses": ["Go to your orders and click 'Cancel'. If it's not shipped yet, we’ll cancel it right away."]
      },
      {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Tell me your name", "Do you have a name?"],
            "responses": [
                "My name is JAMAL, your friendly AI assistant.",
                "You can call me JAMAL — Just A Machine Assisting Life!",
                "I'm JAMAL. I was built by Sam, an AI engineer who loves helping people."
            ]
        },
      {
        "tag": "complaint",
        "patterns": ["I got the wrong item", "My package is damaged", "I want to file a complaint", "This is not what I ordered"],
        "responses": ["I'm really sorry to hear that. Please provide your order ID and we’ll help you fix it.", "Apologies for the inconvenience. Let’s make this right."]
      },
      {
            "tag": "creator",
            "patterns": [
                "Who made you?",
                "Who created you?",
                "Who built you?",
                "Tell me about your creator"
            ],
            "responses": [
                "I was created by Sam, an AI engineer who loves building smart helpers!",
                "Sam is the brilliant mind behind JAMAL. 😊",
                "An awesome engineer named Sam built me to assist you with AI superpowers!"
            ]
        },
      {
        "tag": "account_help",
        "patterns": ["I forgot my password", "Can't log into my account", "How do I reset my password?", "Login issues"],
        "responses": ["You can reset your password using the 'Forgot Password' link on the login page.", "Need help with your account? Try resetting your password first."]
      }
    ]
  }


In [65]:
# 2. Preprocessing
all_patterns = []
all_tags = []
for intent in intents['intents']:
  for pattern in intent['patterns']:
    all_patterns.append(pattern)
    all_tags.append(intent['tag'])

In [66]:
# Clean Text
def clean_text(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = nltk.word_tokenize(text)
  return tokens

tokenizer = Tokenizer(oov_token="<OOV>")
sequences = [clean_text(p) for p in all_patterns]
texts = [" ".join(s) for s in sequences]

In [67]:
# Tokenize and pad
tokenizer.fit_on_texts(texts)
x = tokenizer.texts_to_sequences(texts)
x = pad_sequences(x, padding ='post')

In [68]:
# Encode labels
y_encoder = LabelEncoder()
y = y_encoder.fit_transform(all_tags)
y = tf.keras.utils.to_categorical(y) # get it ready to be used by neural network

In [69]:
# Build LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16, input_length=x.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [70]:
# 4. Train
model.fit(x, y, epochs=200, verbose=1)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.0366 - loss: 2.5655
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1332 - loss: 2.5629
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0862 - loss: 2.5616
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1174 - loss: 2.5596
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1332 - loss: 2.5583
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1801 - loss: 2.5564
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1932 - loss: 2.5548
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1959 - loss: 2.5534
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2610 - loss: 2.5505
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2271 - loss: 2.5480
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2063 - loss: 2.5452
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2271 - lo

In [71]:
# 5. Chatbot response
def get_response(user_input):
  tokens = clean_text(user_input)
  seq = tokenizer.texts_to_sequences([" ".join(tokens)])
  padded = pad_sequences(seq, maxlen=x.shape[1], padding='post')
  pred = model.predict(padded, verbose=0)
  tag = y_encoder.inverse_transform([np.argmax(pred)])
  for intent in intents['intents']:
    if intent['tag'] == tag[0]:
      return random.choice(intent['responses'])

# 6. Interaction Loop
print("Chatbot is ready! Type quit to exit.")
while True:
  inp = input("You: ")
  if inp.lower() == 'quit':
    break
  print("JAMAL:", get_response(inp))

Chatbot is ready! Type quit to exit.
You: hey
JAMAL: Hello! How may I help you?
You: how made you?
JAMAL: Sam is the brilliant mind behind JAMAL. 😊
You: who is Sam?
JAMAL: Sure! Our return policy allows returns within 30 days of delivery.
You: quit.
JAMAL: Goodbye! Have a great day!
You: quit
